# Face Recognizer - Andy Sharpe, Eric Brito, and Bruce Huang

This program has two abilities, the first one is to take an image file and detect if it contains faces, while showing an image showing boxes arround the detected faces and eyes.
The second function is to use the computor's webcam to take images and do the same process in realtime

This is the process that the program goes throught when the cells are ran in order:

Basic setup: (everyone worked together and on the parts pertaining to their specific code)
    1. Install the python version of OpenCV for use in face detection and showing the output window
    2. Imports all of the relevant libraries including cv2, tkinter (and its file selection dialog code), and PIL (only the Image and ImageTK components)
    3. Set up variables that are global or constant
    4. Have the user select the XML files for face and eye detection from their computor (the download links are before the relevant cells)
    5. If this does not work an error should be printed and the user should re-run that cell
    
UI setup: (Andy)
    1. Using tkinter create a window with a title (Face recognizer) and four buttons (Select file, Use webcam, run, and quit)
    2. Setup the buttons to run their respective functions when clicked and disable the run button untill a file is selected properly
    3. If the Quit button or the close button in the window are pressed the close function will be ran to insure proper closeure of the windows and program
    4. If the Select file button is used the program will start on the image face detection path and if the Use webcam button is used the program will start on the webcam realtime face detection path
    
Image face detection: (UI: Andy, Image/Logic programming: Bruge, implementing machine learning: Eric)
    1. Once the Select file button is pressed the user can select a file that is an image type that can work both with tkinter and OpenCV which will then be used to create a comboImage object that contains image data for both formats
    2. The ui then shows this image (either creating it or replacing the previous one) and scales down it to fit the window if it is too big and then re-adjusts the window accordingly
    4. If the Run button is pressed than the checkImage function is started and the saved comboImage object is used with with OpenCV and the previoously selected XML files to find any faces and eyes in the picture
    5. The comboImage object will then display the origional image with boxes superimposed over it showing the detected faces and eyes in blue and green and depending on if a face was found a corrisponding statment will be printed

Webcam realtime face detection: (Image/Logic programming: Bruge, implementing machine learning: Eric)
    1. Once the Use webcam button is pressed the computor's webcam will be turned on and using OpenCV each current frame will be turned into a comboImage object for later use
    2. For the frame a simular process is done as the one above for a single image and the frame will be returned with boxes on it in realtime, which due to the constant frame inputs will give the appearance of a video
    3. If the window is detected as closed than the program will stop taking and showing the webcam images and return to its default state

In [7]:
#RUNNING THE FOLLOWING MAY TAKE A WHILE, DO NOT RUN THE OTHER CELLS UNTILL THIS IS COMPLETE
#this installs the OpenCV library that is used to analyze the images with the machine learning algorythems

In [8]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
#the library from openCV
import cv2
#the library used for the ui
from tkinter import *
#the library used putting/changing images for the ui
from PIL import Image, ImageTk
#allows a file seceltion screen to be used
from tkinter.filedialog import askopenfilename
#sets up some variables
globIm = None
globT = None
userImage = None
startW = 726
startH = 0

The following code will ask for two files to be selected, for the first one select "haarcascade_frontalface_default" and for the second one select "haarcascade_eye"
These are the machine learning algorythems used to detect faces and eyes respecively
Download at https://github.com/ajcres/UCSDProj2/blob/master/haarcascade_frontalface_default.xml and https://github.com/ajcres/UCSDProj2/blob/master/haarcascade_eye.xml
If you want to detect another object then an XML file relating to it can be selected instead

In [2]:
#code that sets up the face recognition code:

#deletes any windows left over from previous runs/errors
try:
    globT.destroy()
except:
    print("",end="")
#sets up file selection window
w1 = Tk()
w1.update()
w1.withdraw()
#brings window to the top
w1.lift()
w1.wm_attributes('-topmost', 1)
#have the user select the face machine learning (XML) file
try:
    faceML = askopenfilename(filetypes=[("XML documents","*.xml")])
    face_cascade = cv2.CascadeClassifier(faceML)
    w1.destroy()
except:
    print("File Selection Error - rerun this cell to correct")
    w1.destroy()
#sets up file selection window
w2 = Tk()
w2.update()
w2.withdraw()
#brings window to the top
w2.lift()
w2.wm_attributes('-topmost', 1)
#have the user select the eyes machine learning (XML) file
try:
    eyeML = askopenfilename(filetypes=[("XML documents","*.xml")])
    eye_cascade = cv2.CascadeClassifier(eyeML)
    w2.destroy()
except:
    print("File Selection Error - rerun this cell to correct")
    w2.destroy()

In [13]:
#class for storing the PIL and cv2 image data and processing it
class comboImage:
    """Since cv2 and tkinter use 2 different image systems, this allows them and their relevant functions to be combined into one class for easy use"""
    global globT
    def __init__ (self,fName):
        """Initializes the comboImage with either a cv2 image or a file location and sets up  its variables"""
        #depending on if the imput is an image or a file name, set up differently
        if type(fName) is str:
            self.pillowIm = Image.open(fName)
            self.cv2Im = cv2.imread(fName)
            self.cam = False
        else:
            #if an image is inputted
            self.cv2Im = fName
            self.cam = True
    def width (self):
        """Returns the width of the image"""
        return self.pillowIm.width
    def height (self):
        """Returns the height of the image"""
        return self.pillowIm.height
    def resize (self,w,h):
        """Changes the dimensions of the non-cv2 image"""
        if w<=0:
            w=self.pillowIm.width
        if h<=0:
            h=self.pillowIm.height
        self.pillowIm = self.pillowIm.resize((w,h),Image.ANTIALIAS)
    def getPILL (self):
        """Returns the image in PIL format"""
        return self.pillowIm
    def getCV (self):
        """Returns the image in cv2 format"""
        return self.cv2Im
    def getFaceImage(self):
        """Finds the faces and eyes in the image and displays the image with boxes arround them"""
        #gets image from the selected image earlier
        img = self.cv2Im
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        if not self.cam: #code if the image ws from a file
            #gets faces useing the XML files form earlier
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            #looks at the faces and eyes and draws boxes arround them
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = img[y:y+h, x:x+w]
                #gets eyes on the faces useing the XML files form earlier
                eyes = eye_cascade.detectMultiScale(roi_gray)
                for (ex,ey,ew,eh) in eyes:
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
            #shows the combination of the image and the boxes
            cv2.destroyAllWindows()
            cv2.imshow('Face detection',img)
            cv2.waitKey(0)
            #if there is more than one face print that one was found
            if len(faces)>0:
                print('face(s) found')
            else:
                print('no face(s) found')
        else: #code if the image ws from the webcam
            #gets faces useing the XML files form earlier
            faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors=5, minSize=(30, 30), flags = cv2.CASCADE_SCALE_IMAGE)
            eyes = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors=5, minSize=(30, 30), flags = cv2.CASCADE_SCALE_IMAGE)
            #looks at the faces and eyes and draws boxes arround them
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = img[y:y+h, x:x+w]
                #gets eyes on the faces useing the XML files form earlier
                eyes = eye_cascade.detectMultiScale(roi_gray)
                for (ex,ey,ew,eh) in eyes:
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            #shows the combination of the image and the boxes
            cv2.imshow('Live feed',img)
        

In [14]:
#image file selection code
def fileSelect():
    """Allows the user to select an image file, whiich then will be added to the ui window and can be used in the 'run' function to detect any faces in it"""
    Tk().withdraw()
    #conects to global variables
    global globIm
    global globT
    global userImage
    try:
        #brings window to the front
        globT.update()
        globT.withdraw()
        globT.lift()
        globT.wm_attributes('-topmost', 0)
        #ask user for an image file that is of a type that can be used with both tkinter and cv2
        #use that file name to create a comboImage
        globIm = comboImage(askopenfilename(filetypes=[("Image Files",("*.png","*.jpeg","*.jp2","*.jpeg","*.jpg"))]))
        
        #sets up image so that it fits in the ui window if it is too big
        if (globIm.width()>600):
            globIm.resize(600,-1)
        if (globIm.height()>400):
            globIm.resize(-1,400)
        
        tempIm = ImageTk.PhotoImage(globIm.pillowIm)
        
        #first time image ui setup: add an image
        if userImage is None:
            userImage = Label(globT,image=tempIm)
            userImage.image = tempIm
            userImage.pack(side=BOTTOM)
        else:
            #if this is not the first time that an image has been selected, just repace the image
            userImage.configure(image=tempIm)
            userImage.image = tempIm
        #allows the "run" button to be pressed
        goButton['state']='normal'
        #brings central ui back up and update it so that the image shows
        globT.deiconify()
        globT.update()
        #set minimum size based on image and sets the window to that siz
        globT.minsize(startW, startH+globIm.height()-100)
        globT.geometry(str(startW)+"x"+str(startH+globIm.height()-100))
        
    except:
        #error code if somthing goes wrong
        print("File type not supported or selection window closed")
        globT.deiconify()
def useCamera():
    """Allows the user to use their computor's camera to detect faces and see the results in realtime"""
    #starts recoding video from the user's camera
    video_capture = cv2.VideoCapture(0)
    ttt = True
    try:
        #loops while the video window is open
        while cv2.getWindowProperty('Live feed', 0) >= 0 or ttt:
            #capture each frame
            ret, frame = video_capture.read()
            
            #creates image for each frame and shows it
            newIm = comboImage(frame)
            newIm.getFaceImage()
            
            ttt=False
            #closes automaticly if the escape key is pressed
            key = cv2.waitKey(20)
            if key == 27:
                break
    except:
        print("Error ocurred")

    #after the window closes, clear everything created in the previous process
    video_capture.release()
    cv2.destroyAllWindows()
    globT.deiconify()
    
def checkImage():
    """Checks if the image selected in the fileSelect function contains a face, and shows an image showing them and any eyes"""
    #gets image from the selected image earlier
    globIm.getFaceImage()
#close windows/code
def close():
    """This functino closes the main ui window (tkinter) and all of the processed image windows (cv2)"""
    cv2.destroyAllWindows()
    globT.destroy()
    globT.quit()

In [16]:
#Run this to start the main program and look for the created window to use

#sets up main ui
globT = Tk()
#resets previous images form memory
userImage = None
#closes any previous cv2 windows
cv2.destroyAllWindows()
#sets the colse function to be called when the ui window is closed
globT.protocol("WM_DELETE_WINDOW", close)
#brings ui to front
globT.update()
globT.lift()
globT.wm_attributes('-topmost', 1)
#makes area for the buttons to be (on the bottom)
f1 = Frame(globT)
f1.pack(side=BOTTOM)
    
#sets up the top label text
topText = Message(globT, text="Face recognizer",width=300,font=('arial', 20))
topText.pack(side=TOP)

#sets up the buttons and adds them to the ui:
#makes the Welect file button run the fileSelect function when pressed and adds it to the ui
selButton = Button(f1, text = "Select file", fg="black",command = fileSelect,width=14,font=('arial', 16))
selButton.pack(side=LEFT)
#makes the Wse webcam button run the useCamera function when pressed and adds it to the ui
camButton = Button(f1, text = "Use webcam", fg="black",command = useCamera,width=14,font=('arial', 16))
camButton.pack(side=LEFT)
#makes the Run button run the checkImage function when pressed and adds it to the ui
goButton = Button(f1, text = "Run", fg="green",command = checkImage,width=14,font=('arial', 16),state='disabled')
goButton.pack(side=LEFT)
#makes the Quit button run the close function when pressed and adds it to the ui
quitButton = Button(f1, text = "Quit", fg="red",command=close,width=14,font=('arial', 16,'bold'))
quitButton.pack(side=LEFT)

#sets up the ui's minimum size based on the size that currently fits everything
globT.minsize(726, globT.winfo_height())
startW = 726
startH = globT.winfo_height()
#runs the ui code
globT.mainloop()

face(s) found
File type not supported or selection window closed
File type not supported or selection window closed
